In [1]:
#| default_exp tbats

# TBATS model 

In [4]:
#| export
import os
import numpy as np 
from numba import njit 
from scipy.optimize import minimize_scalar

In [5]:
#| export 
# Global variables 
NOGIL = os.environ.get('NUMBA_RELEASE_GIL', 'False').lower() in ['true']
CACHE = os.environ.get('NUMBA_CACHE', 'False').lower() in ['true']

## Load data 

In [7]:
#| hide 
# AirPassengers 
from statsforecast.utils import AirPassengers as ap 

## Box-Cox Transformation

In [74]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def guer_cv(lam, x, season_length): 
    
   """Minimize this funtion to find the optimal parameter for the Box-Cox transformation."""
   period = np.round(np.max(season_length)) 
   n = len(x) 
   nyears = int(np.floor(n/period))
   nobs = np.floor(nyears*period)
   m = int(n-nobs)
   xmat = x[m:n].reshape((nyears, period))

   xmean = np.full(xmat.shape[0], fill_value = np.nan)
   for k in range(xmat.shape[0]): 
      xmean[k] = np.nanmean(xmat[k])

   xsd = np.full(xmat.shape[0], fill_value = np.nan)
   for k in range(xmat.shape[0]): 
      vals = xmat[k]
      svar = (vals-np.nanmean(vals))**2 
      svar = np.sum(svar)/len(svar) 
      xsd[k] = np.sqrt(svar) 

   xrat = xsd/(xmean**(1-lam))

   sd = (xrat-np.nanmean(xrat))**2 
   sd = np.nansum(sd)/(len(sd)-1)
   sd = np.sqrt(sd)
   
   return sd/np.nanmean(xrat)

In [75]:
#| exporti
def guerrero(x, season_length, lower=0, upper=1): 
    
    """Finds optimal paramater for Box-Cox transformation using Guerrero's method"""
    
    if np.any(x < 0): 
        raise ValueError("Guerrero's method for selecting a Box-Cox parameter (w) is for strictly positive data")
    
    max_freq = np.max(season_length)
    if len(x) <= 2*max_freq: 
        res = 1
    else: 
        w = 0 # initial guess 
        opt = minimize_scalar(guer_cv, w, args = (x, season_length), method = 'bounded', bounds = (lower, upper))
        res = opt.x

    return res

In [77]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def BoxCox(y, BoxCox_lambda): 
    
    """Applies Box-Cox transformation with parameter BoxCox_lambda"""

    if BoxCox_lambda == 0: 
        z = np.log(y)
    else: 
        z = np.sign(y)*((np.abs(y)**BoxCox_lambda)-1)
        z = z/BoxCox_lambda
        
    return z

In [78]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def InverseBoxCox(z, BoxCox_lambda): 
    
    """Inverts Box-Cox transformation with parameter BoxCox_lambda"""

    if BoxCox_lambda == 0: 
        y = np.exp(z) 
    else: 
        sign = np.sign(BoxCox_lambda*z+1)
        y = np.abs(BoxCox_lambda*z+1)**(1/BoxCox_lambda)
        y = sign*y 
        
    return y 

## Functions 

## TBATS model 